In [1]:
import cv2
import numpy as np
from scipy import ndimage
import scipy
from scipy import ndimage
import imageio
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io
import matplotlib.pyplot as plt
import skimage.filters

In [2]:
def to_shape(a, shape):
    b = np.zeros(shape)
    b[:a.shape[0], :a.shape[1]] = a
    return b



In [3]:
video = cv2.VideoCapture(0)
 
# Infinite while loop to treat stack of image as video
while True:
    check, frame = video.read()
    
    # im = scipy.signal.medfilt2d(im)
    # im = skimage.filters.gaussian(im, sigma=(sigma, sigma), multichannel=True)


    # #sobel operator
    # # Reading frame(image) from video
    # im = frame.astype('int32')
    # # im = frame.astype('int32')
    # dx = ndimage.sobel(im, 0) # horizontal deriv
    # dy = ndimage.sobel(im, 1) # vertical deriv
    # mag = np.hypot(dx, dy) # magnitude
    # mag *= 255.0/np.max(mag) #normalize

    ###############################################
    # Morphological edge out
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    threshold = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.morphologyEx(threshold, cv2.MORPH_DILATE, kernel)
    morph = cv2.absdiff(dilate, threshold)   
    ###############################################



    ###############################################
    # Canny Edge Detection        
    # cv2.Canny(frame_webcam, 100, 110, edges=outframe)
    canny = cv2.Canny(frame, 100, 110)
    ###############################################


    # noise reduction using median blur
    sigma = 7
    # frame2 = cv2.medianBlur(frame, sigma)
    frame2 = cv2.GaussianBlur(frame, (sigma,sigma), 0)
    im = frame2.astype('int32')
    ###############################################
    # Sobel Edge Detection
    # define filters
    horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
    vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1
    gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

    img_fft = np.fft.fft2(gray_img)
    horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
    vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


    newverticalImage = np.fft.ifft2( img_fft * vertical_kernel_fft )
    newhorizontalImage = np.fft.ifft2( img_fft * horizontal_kernel_fft )

    sobel_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)
    # sobel_fft = sobel_fft-25
    sobel_fft[sobel_fft<30] = 0

    ###############################################



    ###############################################
    # Prewitt Edge Detection
    # define filters
    horizontal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # s2
    vertical = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])  # s1
    gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

    img_fft = np.fft.fft2(gray_img)
    horizontal_kernel_fft =  np.fft.fft2(to_shape(horizontal, gray_img.shape))
    vertical_kernel_fft =  np.fft.fft2(to_shape(vertical, gray_img.shape) )


    newverticalImage = np.fft.ifft2( img_fft * vertical_kernel_fft )
    newhorizontalImage = np.fft.ifft2( img_fft * horizontal_kernel_fft )

    prewitt_fft = abs( np.sqrt(pow(newverticalImage, 2) + pow(newhorizontalImage, 2)) ).astype(float)
    # prewitt_fft = prewitt_fft-25
    prewitt_fft[prewitt_fft<30] = 0

    
    ###############################################


    cv2.imshow("normal frame", frame)
    cv2.imshow("edge detection sobel", sobel_fft)
    cv2.imshow("edge detection prewitt", prewitt_fft)
    cv2.imshow("morphological edge out", morph)
    cv2.imshow("canny edge detection", canny)
   
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()


KeyboardInterrupt: 

In [ ]:
# video = cv2.VideoCapture(0)
 
# # Infinite while loop to treat stack of image as video
# while True:
#     # Reading frame(image) from video
#     check, frame = video.read()

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     threshold = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)[1]
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
#     dilate = cv2.morphologyEx(threshold, cv2.MORPH_DILATE, kernel)

#     diff = cv2.absdiff(dilate, threshold)   
#     # edges = 255-diff
            
#     # cv2.Canny(frame_webcam, 100, 110, edges=outframe)
#     cv2.imshow("canny edge detection", diff)
#     cv2.imshow("normal frame", frame)
#     key = cv2.waitKey(1)
#     if key == ord('q'):
#         break
# video.release()

In [ ]:
# video = cv2.VideoCapture(0)
 
# # Infinite while loop to treat stack of image as video
# while True:
#     check, frame = video.read()

#     im = frame
    
#     gray_img = np.round(0.299 * im[:, :, 0] +
#                         0.587 * im[:, :, 1] +
#                         0.114 * im[:, :, 2]).astype(np.uint8)

#     # Sobel Operator
#     h, w = gray_img.shape
#     # define filters
#     horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
#     vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1

#     # define images with 0s
#     newhorizontalImage = np.zeros((h, w))
#     newverticalImage = np.zeros((h, w))
#     newgradientImage_sobel = np.zeros((h, w))

#     # offset by 1
#     for i in range(1, h - 1):
#         for j in range(1, w - 1):
#             horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
#                             (horizontal[0, 1] * gray_img[i - 1, j]) + \
#                             (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
#                             (horizontal[1, 0] * gray_img[i, j - 1]) + \
#                             (horizontal[1, 1] * gray_img[i, j]) + \
#                             (horizontal[1, 2] * gray_img[i, j + 1]) + \
#                             (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
#                             (horizontal[2, 1] * gray_img[i + 1, j]) + \
#                             (horizontal[2, 2] * gray_img[i + 1, j + 1])

#             newhorizontalImage[i - 1, j - 1] = abs(horizontalGrad)

#             verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
#                         (vertical[0, 1] * gray_img[i - 1, j]) + \
#                         (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
#                         (vertical[1, 0] * gray_img[i, j - 1]) + \
#                         (vertical[1, 1] * gray_img[i, j]) + \
#                         (vertical[1, 2] * gray_img[i, j + 1]) + \
#                         (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
#                         (vertical[2, 1] * gray_img[i + 1, j]) + \
#                         (vertical[2, 2] * gray_img[i + 1, j + 1])

#             newverticalImage[i - 1, j - 1] = abs(verticalGrad)

#             # Edge Magnitude
#             mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
#             newgradientImage_sobel[i - 1, j - 1] = mag

#     cv2.imshow("edge detection sobel", newgradientImage_sobel)
#     cv2.imshow("normal frame", frame)
#     key = cv2.waitKey(1)
#     if key == ord('q'):
#         break
# video.release()